### Importando bibliotecas necessárias

In [3]:
import requests
from datetime import datetime
import pandas as pd
import pandas_profiling
import pprint
import json
from IPython.display import display
from concurrent.futures import ThreadPoolExecutor
from elasticsearch import Elasticsearch
from elasticsearch import helpers
es_client = Elasticsearch(http_compress=True)

In [4]:
pp = pprint.PrettyPrinter(indent=4)
pd.set_option('display.max_colwidth',100)

## Processo de ETL

### Etapa de **extração dos dados**

#### Função que recupera a lista de todos os deputados

In [5]:
def listaDeputados(url):
    headers = {'accept': 'application/json'}
    r = requests.get(url, headers=headers)
    return r.json()['dados']

#### Carrega lista de deputados em um dataframe pandas

In [6]:
deputados = pd.DataFrame(listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome'))

In [7]:
deputados.count()

email            513
id               513
idLegislatura    513
nome             513
siglaPartido     513
siglaUf          513
uri              513
uriPartido       513
urlFoto          513
dtype: int64

#### Função que recupera para cada deputado todas as despesas do ano de 2019  


In [10]:
despesas = {}
list_dataframes = []
list_deputados = deputados.head(5)
for index, row in list_deputados.iterrows():
    pagina = 1
    while True:
        url = f"https://dadosabertos.camara.leg.br/api/v2/deputados/{row['id']}/despesas?ano=2019&pagina={pagina}&itens=100&ordem=ASC&ordenarPor=dataDocumento"
        headers = {'accept': 'application/json'}
        r = requests.get(url, headers=headers)
        if not r.json()['dados']:
            break
        desp_dep = pd.DataFrame(r.json()['dados'])
        desp_dep['dep_id'] = row['id']
        desp_dep['dep_nome'] = row['nome']
        desp_dep['dep_partido'] = row['siglaPartido']
        desp_dep['dep_uf'] = row['siglaUf']
        list_dataframes.append(desp_dep)
        pagina = pagina + 1
        update_progress(index/(len(list_deputados)-1))
despesas = pd.concat(list_dataframes)

Progress: [####################################################################################################] 100.0%


### Etapa de **transformação dos dados**  

### Limpeza dos dados coletados
#### Nessa etapa será realizada a limpeza e adequação do dados baseado em uma análise utilizando a biblioteca `pandas_profile` através do comando `despesas.profile_report(style={'full_width':True})`
- Remoção de colunas desnecessárias do DataFrame, colunas que não contém dados ou já possuem representatividade em outra coluna do DataFrame
- Adequação de campos Nulos, inserindo valores padrão nessas posições
- Conversão dos tipos de dados para numerico e data

In [ ]:
despesas.profile_report(style={'full_width':True})

**Excluindo colunas desnecessárias**

In [10]:
despesas.drop(columns=['ano','codLote','parcela','urlDocumento','valorLiquido'],inplace=True)

**Ajustando os tipos de dados**  
Transforma a data para o tipo datetime, caso o campo esteja vazio define a data como sendo o último dia do ano de 2018.

In [39]:
despesas['dataDocumento'] = despesas['dataDocumento'].apply(lambda x: pd.to_datetime(x) if not pd.isna(x) else datetime(2018,12,31,0,0))

In [20]:
despesas.dtypes

cnpjCpfFornecedor            object
codDocumento                  int64
codTipoDocumento              int64
dataDocumento        datetime64[ns]
mes                           int64
nomeFornecedor               object
numDocumento                 object
numRessarcimento             object
tipoDespesa                  object
tipoDocumento                object
valorDocumento              float64
valorGlosa                  float64
dep_id                        int64
dep_nome                     object
dep_partido                  object
dep_uf                       object
dtype: object

In [15]:
despesas['tipoDespesa'].value_counts()

Emissão Bilhete Aéreo                                        7548
COMBUSTÍVEIS E LUBRIFICANTES.                                7263
SERVIÇO DE TÁXI, PEDÁGIO E ESTACIONAMENTO                    3345
MANUTENÇÃO DE ESCRITÓRIO DE APOIO À ATIVIDADE PARLAMENTAR    2522
FORNECIMENTO DE ALIMENTAÇÃO DO PARLAMENTAR                   2319
TELEFONIA                                                    2232
SERVIÇOS POSTAIS                                             1955
DIVULGAÇÃO DA ATIVIDADE PARLAMENTAR.                          908
HOSPEDAGEM ,EXCETO DO PARLAMENTAR NO DISTRITO FEDERAL.        629
LOCAÇÃO OU FRETAMENTO DE VEÍCULOS AUTOMOTORES                 554
PASSAGENS AÉREAS                                              391
CONSULTORIAS, PESQUISAS E TRABALHOS TÉCNICOS.                 221
PASSAGENS TERRESTRES, MARÍTIMAS OU FLUVIAIS                   146
ASSINATURA DE PUBLICAÇÕES                                      83
SERVIÇO DE SEGURANÇA PRESTADO POR EMPRESA ESPECIALIZADA.       61
LOCAÇÃO OU

In [40]:
despesas.isna().sum()

ano                  0
cnpjCpfFornecedor    0
codDocumento         0
codLote              0
codTipoDocumento     0
dataDocumento        0
mes                  0
nomeFornecedor       0
numDocumento         0
numRessarcimento     0
parcela              0
tipoDespesa          0
tipoDocumento        0
urlDocumento         0
valorDocumento       0
valorGlosa           0
valorLiquido         0
dep_id               0
dep_nome             0
dep_partido          0
dep_uf               0
dtype: int64

In [37]:
despesas.info(memory_usage='deep')

<class 'pandas.core.frame.DataFrame'>
Int64Index: 716 entries, 0 to 1
Data columns (total 21 columns):
ano                  716 non-null int64
cnpjCpfFornecedor    716 non-null object
codDocumento         716 non-null int64
codLote              716 non-null int64
codTipoDocumento     716 non-null int64
dataDocumento        691 non-null object
mes                  716 non-null int64
nomeFornecedor       716 non-null object
numDocumento         716 non-null object
numRessarcimento     716 non-null object
parcela              716 non-null int64
tipoDespesa          716 non-null object
tipoDocumento        716 non-null object
urlDocumento         716 non-null object
valorDocumento       716 non-null float64
valorGlosa           716 non-null float64
valorLiquido         716 non-null float64
dep_id               716 non-null int64
dep_nome             716 non-null object
dep_partido          716 non-null object
dep_uf               716 non-null object
dtypes: float64(3), int64(7), object(11)

### Carga dos dados para o banco elasticsearch

In [30]:
use_these_keys = despesas.columns.to_list()
def filterKeys(document):
    return {key: document[key] for key in use_these_keys }

In [31]:
def doc_generator(df):
    df_iter = df.iterrows()
    for index, document in df_iter:
        yield {
                "_index": 'teste',
                "_type": "_doc",
                "_source": filterKeys(document),
            }
    raise StopIteration

In [32]:
try:
    helpers.bulk(es_client, doc_generator(despesas))
except:
    print('Fim do carregamento para o Elasticsearch')

Fim do carregamento para o Elasticsearch


In [ ]:
for i in doc_generator(despesas):
    helpers.bulk(es_client, i)
print('Fim do carregamento para o Elasticsearch')

In [7]:
es_client.indices.create(index='test-index', ignore=400)

{'acknowledged': True, 'shards_acknowledged': True, 'index': 'test-index'}

### Funções auxiliares

In [9]:
import time, sys
from IPython.display import clear_output

def update_progress(progress):
    bar_length = 100
    if isinstance(progress, int):
        progress = float(progress)
    if not isinstance(progress, float):
        progress = 0
    if progress < 0:
        progress = 0
    if progress >= 1:
        progress = 1

    block = int(round(bar_length * progress))

    clear_output(wait = True)
    text = "Progress: [{0}] {1:.1f}%".format( "#" * block + "-" * (bar_length - block), progress * 100)
    print(text)

### Testes

In [ ]:
despesas.loc[despesas['dep_nome'].str.contains('AB')][['tipoDespesa','dataDocumento','dep_nome','valorDocumento']].groupby(by=['dep_nome','tipoDespesa']).agg(['sum','count','mean'])

In [ ]:
despesas.loc[despesas['dep_nome'].str.contains('AB'),['tipoDespesa','dataDocumento','dep_nome','valorDocumento']].groupby(by=['dep_nome','tipoDespesa']).agg(['sum','count','mean'])

In [ ]:
deputados.loc[deputados['nome'].str.contains('Tabata.*', regex=True, case=False)]['uri']

In [ ]:
listaDeputados('https://dadosabertos.camara.leg.br/api/v2/deputados?ordem=ASC&ordenarPor=nome')

In [ ]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].count().sort_values(ascending=False)

In [ ]:
despesas[(despesas['tipoDespesa'].str.contains('COMBUSTÍVEIS'))].groupby(by=['nomeFornecedor'])['valorDocumento'].sum().sort_values(ascending=False)

In [ ]:
despesas.loc[(despesas['nomeFornecedor'].str.contains('CASCOL COMBUSTIVEIS'))]

In [ ]:
despesas[(despesas['cnpjCpfFornecedor']=="17895646000187")].head()